<a href="https://colab.research.google.com/github/csaybar/EarthEngineMasterGIS/blob/master/module04/02_EDGEE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!--COURSE_INFORMATION-->
<img align="left" style="padding-right:10px;" src="https://user-images.githubusercontent.com/16768318/73986808-75b3ca00-4936-11ea-90f1-3a6c352766ce.png" width=10% >
<img align="right" style="padding-left:10px;" src="https://user-images.githubusercontent.com/16768318/73986811-764c6080-4936-11ea-9653-a3eacc47caed.png" width=10% >

**Bienvenidos!** Este *colab notebook* es parte del curso [**Introduccion a Google Earth Engine con Python**](https://github.com/csaybar/EarthEngineMasterGIS) desarrollado por el equipo [**MasterGIS**](https://www.mastergis.com/). Obten mas informacion del curso en este [**enlace**](https://www.mastergis.com/product/google-earth-engine/). El contenido del curso esta disponible en [**GitHub**](https://github.com/csaybar/EarthEngineMasterGIS) bajo licencia [**MIT**](https://opensource.org/licenses/MIT).

## **MASTERGIS: ED espaciales en Google Earth Engine I**
## **ee.Image**

En esta lectura, aprenderemos sobre:

- ee.Image como ED nativa de GEE.
- Expresiones matematicas (indices espectrales).
- Como exportar una imagen a google drive

### **1) Autenticar y inicializar GEE**

In [0]:
#@title Credenciales Google Earth Engine
import os 
credential = '{"refresh_token":"PON_AQUI_TU_TOKEN"}'
credential_file_path = os.path.expanduser("~/.config/earthengine/")
os.makedirs(credential_file_path,exist_ok=True)
with open(credential_file_path + 'credentials', 'w') as file:
    file.write(credential)

In [0]:
import ee
ee.Initialize()

### **2) Carga nuestra funcion de mapeo**

In [0]:
#@title mapdisplay: Crea mapas interactivos usando folium
import folium
def mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    center = center[::-1]
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

### **3. Images en GEE (ee.Image)**

> Imagen: Bandas + Metadatos (JSON)


Ademas de cargar imagenes del catalogo de datos de GEE mediante una **ID de imagen**, tambien puede crear imagenes a partir de constantes, listas u otros objetos de Earth Engine. Esta seccion ilustra los metodos para crear imagenes, obtener subconjuntos de bandas y manipular bandas.

Metodos que usara en esta seccion:

  - **ee.Image.cat**
  - **ee.Image.addBands**
  - **ee.Image.rename**
  - **ee.Image.Select**

In [4]:
# 3.1 Crear una imagen constante
image1 = ee.Image(1)
print(image1,'\n')
image1.getInfo()

ee.Image({
  "type": "Invocation",
  "arguments": {
    "value": 1
  },
  "functionName": "Image.constant"
}) 



{'bands': [{'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0],
   'data_type': {'max': 1, 'min': 1, 'precision': 'int', 'type': 'PixelType'},
   'id': 'constant'}],
 'type': 'Image'}

- **ee.Image.cat (lista)**: Concatenar las imágenes dadas en una sola imagen.

In [0]:
#3.2 Concatenar dos imagenes en una imagen multibanda.
image2 = ee.Image(2)
image3 = ee.Image.cat([image1, image2])
print(image3,'\n')
image3.getInfo()

In [0]:
# Cree una imagen multibanda a partir de una lista de constantes.
multiband = ee.Image([1, 2, 3])
print(multiband,'\n')
multiband.getInfo()

- **ee.Image.addBands**: Devuelve una imagen que contiene todas las bandas de la primera entrada y las bandas seleccionadas de la segunda entrada, opcionalmente sobrescribiendo las bandas en la primera imagen con el mismo nombre. La nueva imagen tiene los **metadatos** y la huella de la primera imagen de entrada.

In [0]:
# Agregar bandas a una imagen.
image4 = image3.addBands(ee.Image(42))
print(image4)

In [0]:
image4.getInfo()

- **ee.Image.rename**: Nos permite renombrar las bandas.




In [0]:
image4.bandNames().getInfo()
image5 = image4.rename(['b1','b2','b3'])
image5.bandNames().getInfo()

['b1', 'b2', 'b3']

- **ee.Image.select**: Selecciona las bandas de interes.


In [0]:
image6 = image5.select(['b3'])
image6.bandNames().getInfo()

['b3']

### **4. Indices espectrales**

In [0]:
img = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')
centroid = img.geometry().centroid().getInfo()['coordinates']
img.bandNames().getInfo()

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'BQA']

In [0]:
mydicc = {'landsat8': img.getMapId({'min':0,'max':0.5,'bands':['B5','B4','B3']})}
mapdisplay(centroid, mydicc, Tiles="OpensTreetMap",zoom_start=9)

In [0]:
#@title **normalized difference vegetation index (NDVI)**
from IPython.display import Math, HTML
display(HTML("<script src='https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.3/"
               "latest.js?config=default'></script>"))

Math(r'NDVI=\frac{(NIR-RED)}{(NIR+RED)}')



<IPython.core.display.Math object>

In [0]:
nir = img.select('B5')
red = img.select('B4')
ndvi_01 = nir.subtract(red).divide(nir.add(red)).rename('NDVI')

In [0]:
ndvi_02 = img.normalizedDifference(['B5', 'B4']).rename('NDVI')

In [0]:
ndvi_palette = ["#051852", "#FFFFFF", "#C7B59B", "#A8B255", "#A3C020", "#76AD00","#429001", "#006400", "#003B00", "#000000"]
mydicc = {
    'ndvi-method01': ndvi_01.getMapId({'min':-0.1,'max':0.8,'palette':ndvi_palette}),
    'ndvi-method02': ndvi_02.getMapId({'min':-0.1,'max':0.8,'palette':ndvi_palette}),
}
mapdisplay(centroid, mydicc, Tiles="OpensTreetMap",zoom_start=9)

In [0]:
#@title Normalized Difference Water Index (NDWI):
from IPython.display import Math, HTML
display(HTML("<script src='https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.3/"
               "latest.js?config=default'></script>"))

Math(r'NDBI=\frac{(NIR-SWIR)}{(NIR+SWIR)}')

<IPython.core.display.Math object>

In [0]:
ndwi = img.normalizedDifference(['B5', 'B6']).rename('NDBI')

In [0]:
bu_palette = ["#000180", "#0075FD", "#6CFB93", "#F99D05", "#A70700"]
mydicc = {
    'ndwi': ndwi.getMapId({'min':0,'max':1,'palette':bu_palette}),    
}
mapdisplay(centroid, mydicc, Tiles="OpensTreetMap",zoom_start=9)

### **5. Operadores Google Earth Engine**
 _ | _ | GEE
--------------|-------------|-------------
Arithmetic	|+ - * / % **	|add, subtract, multiply, divide, mod, exp
Comparison	| == != < > <= >= |	eq, neq, lt, gt, lte,gte.
Logical |	&& \|\| ! ^	| And, Or, Not, Xor

### **6. Mascaras**

Puede usar **`image.updateMask`** para generar una mascara en pixeles no deseados. Los pixeles iguales a cero en la mascara se excluyen de los calculos y la opacidad se establece en 0 para la visualizacion. El siguiente ejemplo utiliza un umbral basado en el NDWI.

In [0]:
water_mask = ndwi.gte(0.4)
water_mask_mapid = water_mask.getMapId()
mapdisplay(centroid,{'NDWI masked':water_mask_mapid})

In [0]:
ndwiMasked = ndwi.updateMask(water_mask)
ndwiId = ndwiMasked.getMapId({'min': 0.5, 'max': 1, 'palette': ['00FFFF', '0000FF']})

# Display the map with folium!
mapdisplay(centroid,{'NDWI masked':ndwiId})

### **8. Cortar una imagen**
El metodo **`image.clip()`** nos permite cortar image, es util para lograr efectos cartograficos. El siguiente ejemplo recorta la imagen **ndwiMasked**.

In [0]:
# Crea un círculo dibujando un búfer de 20000 metros alrededor de un punto.
roi = ee.Geometry.Point([-122.4481, 37.7599]).buffer(20000)

ndwiMasked_roi = ndwiMasked.clip(roi)
ndwiroiId = ndwiMasked_roi.getMapId({'min': 0.5, 'max': 1, 'palette': ['00FFFF', '0000FF']})

# Display map
center=[38., -122.5]
mapdisplay(centroid,{'Mosaic':ndwiroiId})

### **7. Como exportar una imagen**

In [0]:
from ee.batch import Export

task = Export.image.toDrive(
    image=ndwiMasked.multiply(1000).toInt(),
    description= 'NDWI_MSK.tif',
    folder='MASTERGIS_GEE',    
    scale=30
    #region: geometry.getInfo()['coordinates']
)

task.start()

**Monitoreo de tareas**

In [0]:
import time 
def ee_monitoring(ee_task):
  while ee_task.active():
    print('Sondeo de la tarea (id: {}).'.format(ee_task.id))
    time.sleep(5)

In [0]:
ee_monitoring(task)

### **¿Dudas con este Jupyer-Notebook?**

Estaremos felices de ayudarte!. Create una cuenta Github si es que no la tienes, luego detalla tu problema ampliamente en: https://github.com/csaybar/EarthEngineMasterGIS/issues

**Tienes que dar clic en el boton verde!**

<center>
<img src="https://user-images.githubusercontent.com/16768318/79680748-d5511000-81d8-11ea-9f89-44bd010adf69.png" width = 70%>
</center>